## 데이터 확인
네이버 금융에서 "삼성전자"의 뉴스/공시 제목을 스크래핑합니다.  
https://finance.naver.com/item/news.nhn?code=005930

<img src="image/00.png" width=500 height=300 />

웹브라우져에 보이는 내용을 requests의 get 메서드로 가져올 수 있는지 확인해 봅시다. 다음 코드를 실행하면 웹서버가 반환한 HTML 코드가 출력됩니다. 찾기 (Ctrl + F) 기능으로 위 그림에 포함된 데이터가 있는지 찾아보세요. 예를 들어 "프리미엄" 이라는 키워드를 찾을 수 있나요?  

In [5]:
import requests

url = "https://finance.naver.com/item/news.nhn?code=005930"
resp = requests.get(url)

# 텍스트 안에 뉴스 및 공시 제목에 포함된 문자열이 들어 있는지 확인
print("프리미엄" in resp.text)
print("반도체 제외" in resp.text)

False
False


HTML 코드를 print 해서 확인해 보면 HTML 안에 신문 제목이 없는 것을 알 수 있습니다. HTML 코드에 데이터가 있다면 크롬 개발자 도구에서 Selector를 복사한 후 스크래핑을 끝내면 될텐데, get 메서드가 반환하는 결과에 데이터가 존재하지 않아 추가 분석이 필요합니다. 

주소(https://finance.naver.com/item/news.nhn?code=005930)도 변하지 않고 표시되는 데이터만 변하는 것으로 보아 웹 서버와 여러번 데이터를 주고 받을 것임을 알 수 있습니다. 
웹 서버와 클라이언트(브라우져)가 주고 받는 데이터를 확인하기 위해 크롬 개발자 도구(F12)를 실행합니다. 
Network Tab으로 이동한 후, 웹 페이지의 있는 뉴스/공시 버튼을 클릭합니다.
Network Tab을 확인하면 클라이언트가 서버와 상당히 많은 요청/응답을 주고 받은 것을 알 수 있습니다.

<img src="image/01.png" width=500 height=300 />

## 데이터 분석
첫 번째 신문 기사의 일부를 검색해서 해당 데이터가 포함된 Requests와 Response를 찾습니다. 검색 결과를 보면 news_news.nhn 이름의 requests 140 라인에 데이터가 포함돼 있다는 것을 알 수 있습니다. 검색 결과를 더블 클릭해서 requests 내용을 살펴봅니다.

<img src="image/02.png" width=350 height=250>

Get 프로토콜을 사용한 다는 것을 알 수 있으며, Get 프로토콜 이므로 URL에 데이터가 포함돼 있습니다. 

<div style="width:350px; height:250px; float:left; margin-right:30px">
    <img src="image/03.png" width=100% height=100% />    
</div>
<div style="width:350px; height:250px; float:left;">
    <img src="image/04.png" width=100% height=100% />    
</div>

해당하는 요청을 웹 서버로 전송하면 웹 서버는 Response Tab에 있는 다음 HTML을 반환할 겁니다. 140 라인의 끝에 보면 검색했던 신문기사가 `<a>` 태그안에 들어 있는것을 확인할 수 있습니다.  
- 삼성·LG 프리미엄 TV 시장점유율 80% 육박..."TV 시장 하반기...

<img src="image/05.png" width=350 height=180>

## 데이터 요청 (Requests) 코드
requests 모듈을 사용해서 Get 프로토콜로 데이터를 요청하는 코드입니다. Headers를 딕셔너리로 작성해서 get 메서드의 headers 파라미터로 전달해도 데이터를 정상적으로 얻어옵니다. 다만 headers를 제거해도 정상 동작하는 것으로 보아 네이버 웹서버는 headers 값을 검사하지 않는 것을 알 수 있습니다. 

In [7]:
import requests

url = "https://finance.naver.com/item/news_news.nhn?code=005930&page=&sm=title_entity_id.basic&clusterId="

resp = requests.get(url)
# 텍스트 안에 뉴스 및 공시 제목에 포함된 문자열이 들어 있는지 확인
print(resp)
print("프리미엄" in resp.text)
print("반도체 제외" in resp.text)

<Response [200]>
True
True


Response Tab 안에서는 CSS Selector를 복사하는 기능이 제공되지 않아 직접 Selector를 작성해야 합니다. 데이터는 tbody 안에 tr 안에 title 클래스를 갖는 td 태그 안에 a 태그에 들어 있습니다. 이를 Selector로 표현하면 다음과 같습니다.

`selector = "tbody > tr > td.title > a"`

## 스크래핑 코드
웹 서버가 HTML 데이터를 반환하기 때문에 Beautiful Soup으로 데이터를 선택합니다. 

In [13]:
from bs4 import BeautifulSoup

bs = BeautifulSoup(resp.text, "html5lib")
selector = "tbody > tr > td.title > a"
result = bs.select(selector)
for r in result:
    print(r.text)

삼성·LG 프리미엄 TV 시장점유율 80% 육박..."TV 시장 하반기...
"삼성전자, 2분기 반도체 제외 전 사업부 수익성 악화"-IBK투자증권
콩나물 에어팟 잡을 '콩' 버즈…무선이어폰도 애플 vs 삼성
"삼성전자, 2분기 말부터 반등"-메리츠증권
갤럭시S20 지원금 최대50만원...삼성이 지갑 연 이유
삼성전자, 그토록 원하는 '시스템반도체'도 세계 1위 가능할까?
삼성 스마트폰, 공들이던 인도서 샤오미·비보에도 밀렸다
먼지통 자동으로 비워주는 삼성 무선청소기 판매 3배↑
시간당 24억씩…'역대 최대' 삼성 R&D 투자 어디에 썼나
삼성전자, 맞춤 디자인 비스포크…색상 10가지 확장한 이유
